In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Create an helper function to import an image and resize it to be able to be used with our model

def load_and_prep_image(filename, img_shape=224):

  # read in the image
  img = tf.io.read_file(filename)
  # Decode the read file into a tensor
  img = tf.image.decode_image(img)
  # resize the image
  img = tf.image.resize(img, size = [img_shape, img_shape])
  # rescale the image
  img = img/255.
  
  return img

In [ ]:
def pred_and_plot(model, filename, class_names):
  
  """
  Imports an image located at filename, makes a prediction with model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  pred_class = class_names[int(tf.round(pred))]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

**Note:** Yukarıdaki 2 fonksiyon beraber kullanılmalıdırlar!

In [ ]:
### Data "augmentation" (example)
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Get the data

import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

# unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

train_dir = "/content/pizza_steak/train"

train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range = 0.2,
                                             shear_range = 0.2,
                                             zoom_range = 0.2,
                                             width_shift_range = 0.2,
                                             height_shift_range = 0.2,
                                             horizontal_flip = True)

train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                   target_size = (224,224),
                                                                   batch_size = 32,
                                                                   class_mode = "binary",
                                                                   shuffle = True) # If we want shuffle adjustments, we do that in here...

--2021-07-05 16:49:23--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.112, 142.250.73.208, 142.250.73.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109579078 (105M) [application/zip]
Saving to: ‘pizza_steak.zip.1’

pizza_steak.zip.1   100%[===================>] 104.50M   207MB/s    in 0.5s    

2021-07-05 16:49:24 (207 MB/s) - ‘pizza_steak.zip.1’ saved [109579078/109579078]

Found 1500 images belonging to 2 classes.


# Callbacks

In [ ]:
# Create a TensorBoard callback (we'll functionize this callback as we'll use it later on again and again)

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
# Making a create_model() function to create a model from a URL

def create_model(model_url, num_classes = 10):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in the output layer, should be equal to the number of target classes, default 10.
  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor layer and Dense output layer with num_classes output neurons.
  """
  # Download the pretrained model and save it as a Keras layer

  feature_extraction_layer = hub.KerasLayer(model_url,
                                           trainable=False, #Freeze the already learned patterns.
                                           name = "feature_extraction_layer",
                                           input_shape = IMAGE_SHAPE+(3,)) 
  
  # Creating the model
  model = tf.keras.models.Sequential([
    feature_extraction_layer,
    layers.Dense(num_classes, activation="softmax", name = "output_layer")
  ])

  return model

In [ ]:
# Function to plot the loss curves

def plot_loss_curves(history):
  
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]
  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]
  epochs = range(len(history.history["loss"]))

  # Plot loss
  plt.plot(epochs, loss, label="training loss")
  plt.plot(epochs, val_loss, label="validation loss")
  plt.title("loss")
  plt.xlabel("epochs")
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label="training accuracy")
  plt.plot(epochs, val_accuracy, label="validation accuracy")
  plt.title("accuracy")
  plt.xlabel("epochs")
  plt.legend;

In [ ]:
# Creating a function to compare training histories

def compare_histories(original_history, new_history, initial_epoch = 5):
  # Get original history measurements
  acc = original_history.history["accuracy"]
  loss = original_history.history["loss"]

  val_acc = original_history.history["val_accuracy"]
  val_loss = original_history.history["val_loss"]

  # Combine original history metrics with new_history metrics
  total_acc = acc + new_history.history["accuracy"]
  total_loss = loss + new_history.history["loss"]

  total_val_acc = val_acc + new_history.history["val_accuracy"]
  total_val_loss = val_loss + new_history.history["val_loss"]

  # Make plots
  plt.figure(figsize = (8,8))
  plt.subplot(2, 1, 1)
  plt.plot(total_acc, label = "Training Accuracy")
  plt.plot(total_val_acc, label = "Validation Accuracy")
  plt.plot([initial_epoch - 1, initial_epoch - 1], plt.ylim(), label = "Start Fine Tuning")
  plt.legend(loc="lower right")
  plt.title("Training and validation accuracy")

  plt.subplot(2, 1, 2)
  plt.plot(total_loss, label = "Training Loss")
  plt.plot(total_val_loss, label = "Validation Loss")
  plt.plot([initial_epoch - 1, initial_epoch - 1], plt.ylim(), label = "Start Fine Tuning")
  plt.legend(loc="upper right")
  plt.title("Training and validation Loss")